# to make the match for the Table 2 1993 QS fe_13 data

## this notebook recreates the basic data used in 

## Dere, 2020, MNRAS, 496, 2334

In [23]:
import json
import pickle
import fnmatch
import matplotlib.pyplot as plt
import numpy as np
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm
import ChiantiPy.Gui as chGui

In [2]:
%autoreload 2

In [3]:
matplotlib qt

In [4]:
myIon = 'Fe XIII'

In [5]:
ls

1_fe_13_demo_make_match.ipynb*  QuickStart.html*
fe_13_demo_chi2.ipynb*          QuickStart.ipynb*
fe_13_demo_make_model.html      README.txt*
fe_13_demo_make_model.ipynb*    tab2_1993_qs_fe_13.json*
fe_13_demo_mcmc.ipynb*


## fe_13 data is in the .json file

In [6]:
with open('tab2_1993_qs_fe_13.json', 'r') as inpt:
    tab = json.load(inpt)

In [7]:
tab.keys()

dict_keys(['filename', 'wvl0', 'intensity', 'intStd', 'dwvl', 'ions', 'ref'])

In [8]:
tab['ref']

['Brosius, Jeffrey W.; Davila, Joseph M.; Thomas, Roger J.; Monsignori-Fossi, Brunella C.',
 'Measuring Active and Quiet-Sun Coronal Plasma Properties with Extreme-Ultraviolet Spectra from SERTS ',
 '1996, Astrophysical Journal Supplement v.106, p.143',
 'ADSref:  https://ui.adsabs.harvard.edu/abs/1996ApJS..106..143B/abstract']

In [9]:
temp = 1.78e+6
dens  = 10.**(6.+ 0.025*np.arange(241))

In [13]:
box = mm.maker(tab, temp, dens, wghtFactor = 0.2, ionList=['fe_13'], verbose = True)

 XUVTOP = /data2/a2git/chdbase/
 abundanceName = sun_photospheric_2015_scott
 minimum abundance =   1.00e+01
 # of observables / reduce # = 9        3.00


## the next step matches the observed lines to the CHIANTI lines to with tab['dwvl']

In [14]:
box.makeMatch(verbose=1)

 fe_13 in the CHIANTI database
 in makeMatchGate thision = fe_13


## next, calculate the contribution functions (gofnt) for each line, including any blends

In [15]:
box.gofnt(temp, dens, verbose=True)

 temperature size:    241
 density     size:    241
 using fe_13
 elapsed seconds =       20.000


## save the match attribute as a pickle file to be picked on again.

In [17]:
matchFilename = 'tab2_1993_qs_fe_13_match.pkl'

In [18]:
box.dumpMatch(filename=matchFilename )

NT not available


In [19]:
ls

1_fe_13_demo_make_match.ipynb*  QuickStart.html*
fe_13_demo_chi2.ipynb*          QuickStart.ipynb*
fe_13_demo_make_model.html      README.txt*
fe_13_demo_make_model.ipynb*    tab2_1993_qs_fe_13.json*
fe_13_demo_mcmc.ipynb*          tab2_1993_qs_fe_13_match.pkl


## the pickle file has already been created

In [24]:
mydir = os.getcwd()
mylist = os.listdir('.')
newlist = fnmatch.filter(mylist,'*.pkl')
selected = chGui.gui.selectorDialog(newlist,'pick one')

In [25]:
mypkl = selected.selectedText[0]

In [26]:
print(mypkl)

tab2_1993_qs_fe_13_match.pkl


## the next two steps if you want to restart without recalculating the match attribute

In [16]:
box = mm.maker(tab)

 XUVTOP = /data2/svn/chianti/dbase/trunk
 abundanceName = sun_photospheric_2015_scott
 minimum abundance =   1.00e-06
 # of observables / reduce # = 9        3.00


In [17]:
box.load(mypkl)

## can just continue here

## plot the contribution functions vs density

In [27]:
temp = box.Temperature
' T:  %12.2e'%(temp[0])

' T:      1.78e+06'

In [28]:
dens = box.EDensity
' Ne:  %12.2e'%(dens[0])

' Ne:      1.00e+06'

In [29]:
nMatches = len(box.match)
print('%3i'%(nMatches))

  9


In [30]:
adjust = np.ones(nMatches,np.float64)

In [31]:
adjust[2] = 0.70
adjust[3] = 1.05
adjust[4] = 0.70
adjust[8] = 0.75

In [32]:
lr = ['l','l','l','r','l','r','r','l','r']

In [33]:
print(' len of lr:  %5i'%(len(lr)))

 len of lr:      9


In [34]:
plt.figure()
box.emPlot(vs='d', fs=12, loc='center right', adjust = adjust, position = lr)

 position is the right length
 0  311.6
 1  312.2
 2  312.9
 3  318.1
 4  320.8
 5  321.5
 6  348.2
 7  359.7
 8  359.9


In [35]:
left,right = plt.xlim()
bottom, top = plt.ylim()
plt.xlim(left, 1.e+14)
plt.ylim(bottom, 1.e+28)
plt.text(1.e+8, 3.e+26, myIon, fontsize=18)
plt.tight_layout()